In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
#from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import nltk
nltk.download('punkt')
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import pipeline, Conversation


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mouaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

In [4]:
llm_loader = PyPDFLoader("rag.pdf")
pages = llm_loader.load_and_split()
text_splitter = CharacterTextSplitter(
separator="separatoor",
is_separator_regex=False,
# Utilisez le séparateur de votre choix
chunk_size=200,
chunk_overlap=0
)
documents=["".join([pages[i].page_content for i in range(len(pages))])]
metadatas = [ {"document":"a propos de djezzy"}]

tokens_chunks = text_splitter.create_documents(
documents, metadatas=metadatas
)

docs_text = [ chunk.page_content for chunk in tokens_chunks ]
docs_embeddings = embedding_llm.embed_documents(docs_text)
query_text = "what are the benefits for Hadj and Omra service 2,000 DA Package?"
query_embedding = embedding_llm.embed_query(query_text)
query_embedding_array = np.array(query_embedding)
docs_embeddings=np.array(docs_embeddings)
similarities = [cosine_similarity(doc.reshape(1,-1), query_embedding_array.reshape(1,-1)) for doc in docs_embeddings]
sorted_docs = sorted(zip(docs_text, docs_embeddings, similarities), key=lambda x: x[2], reverse=True)
similar_docs = [(doc,sim) for doc, _, sim in sorted_docs ]
# Filtrer les documents avec une similarité supérieure à 0.4
similar_docs1 = [(doc,sim) for doc, _, sim in sorted_docs if sim > 0.72]

Created a chunk of size 1542, which is longer than the specified 200
Created a chunk of size 818, which is longer than the specified 200
Created a chunk of size 294, which is longer than the specified 200
Created a chunk of size 257, which is longer than the specified 200
Created a chunk of size 1157, which is longer than the specified 200
Created a chunk of size 1055, which is longer than the specified 200
Created a chunk of size 637, which is longer than the specified 200
Created a chunk of size 625, which is longer than the specified 200
Created a chunk of size 603, which is longer than the specified 200
Created a chunk of size 820, which is longer than the specified 200
Created a chunk of size 501, which is longer than the specified 200
Created a chunk of size 763, which is longer than the specified 200
Created a chunk of size 425, which is longer than the specified 200
Created a chunk of size 706, which is longer than the specified 200
Created a chunk of size 556, which is longer 

In [22]:
#device_map = {"": 0}
from huggingface_hub import notebook_login
notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [20]:
model_name = "google/gemma-2b"
#
## Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,is_decoder=True)
#
## Charger le modèle
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True,is_decoder=True)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b.
401 Client Error. (Request ID: Root=1-664b79e9-37c3d12022890a590782aa85;99e63b7e-c378-4d2e-82ce-7b239f58ee01)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must be authenticated to access it.

In [19]:
context="\n---------------------\n".join([doc for doc,_ in similar_docs[:3]]if len(similar_docs) >=3 else [doc for doc, _ in similar_docs[:1]])
prompt = f" answer the question:{query_text} using this context:{context} " # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000, do_sample= True)
result = pipe(prompt)
print(result[0]['generated_text'])

IndexError: index out of range in self